<a href="https://colab.research.google.com/github/gptix/DS-Unit-2-Linear-Models/blob/master/Jud_Taylor_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

# Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.



## Import useful code.

In [0]:
# Import userful code.
import sys
import warnings
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.express as px

## Import raw data.

In [0]:
# Import data.

# If on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
# If local:
else:
    DATA_PATH = '../data/'
    
# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

## Preliminary review of data.

In [12]:
# Review data.
# df
# df.describe()
# df.head()
# df.tail()
# df.columns
# df.dtypes
# df['bedrooms'].describe()
# df.shape
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space'],
      dtype='object')

## Clean data.

In [0]:
# Remove outliers: 

# the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= 1375) & (df['price'] <= 15500) & 
        (df['latitude'] >=40.57) & (df['latitude'] < 40.99) &
        (df['longitude'] >= -74.1) & (df['longitude'] <= -73.38)]
df.shape

(48818, 34)

## Split for Training & Test.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [0]:
train = df[(df['created'] >= '2016-04') & (df['created'] < '2016-06')]
test  = df[(df['created'] >= '2016-06') & (df['created'] < '2016-07')]
# test['dining_room'].unique()

## Engineer Features.

- [ ] Engineer at least two new features. (See below for explanation & ideas.)

In [0]:
def add_bed_bath_dining_column(frame):
  '''Return a dataframe that has an added feature: the sum o the counts of 
  bedrooms, bathrooms, and dining_room.'''
  frame['bed_bath_dining'] = frame.apply(lambda row: row.bathrooms + row.bedrooms + row.dining_room , axis=1)
  return frame

test  = add_bed_bath_dining_column(test)
train = add_bed_bath_dining_column(train)

# test['bed_bath_dining']

In [18]:
# private_outdoor_fields = ['balcony', 'terrace', 'garden_patio']

def add_private_outdoor(frame):
  frame['private_outdoor'] = frame.apply(lambda row: row.balcony | row.terrace | row.garden_patio, axis=1)
  return frame

test  = add_private_outdoor(test)
train = add_private_outdoor(train)

# test['private_outdoor']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Fit a model with at least two features.
- [ ] Fit a linear regression model with at least two features.


In [0]:
# VDP step one: Select model.
# LinearRegression imported above

In [0]:
# VDP step two: Instantiate model class.
model = LinearRegression()

In [21]:
# VDP step 3: Arrange X features matrices, and y target vectors.
# X
features = ['private_outdoor', 'bed_bath_dining']
X_train = train[features]
X_test = test[features]

# y
y_train = train['price']
y_test = test['price']

print(f'Linear Regression, dependent on: {features}')

Linear Regression, dependent on: ['private_outdoor', 'bed_bath_dining']


In [27]:
# VDP step 4: Fit the model.
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Training set mean absolute error: ${mae:.2f}')

Training set mean absolute error: $894.27


## Get Coefficients and Intercept to use in evaluating model.

- [ ] Get the model's coefficients and intercept.

In [28]:
coefficients = model.coef_
coefficients

array([265.72740142, 754.02827215])

In [29]:
intercept = model.intercept_
intercept

1410.4784720573466

## Get newly introduced metrics.


- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.


### Ryan's code (not used yet)

In [0]:
# Code by Ryan Herr (Lambda School)
# from matplotlib.patches import Rectangle
# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# def squared_errors(df, feature, target, m, b):
#     """
#     Visualize linear regression, with squared errors,
#     in 2D: 1 feature + 1 target.
    
#     Use the m & b parameters to "fit the model" manually.
    
#     df : Pandas DataFrame
#     feature : string, feature column in df
#     target : string, target column in df
#     m : numeric, slope for linear equation
#     b : numeric, intercept for linear requation
#     """
    
#     # Plot data
#     fig = plt.figure(figsize=(7,7))
#     ax = plt.axes()
#     df.plot.scatter(feature, target, ax=ax)
    
#     # Make predictions
#     x = df[feature]
#     y = df[target]
#     y_pred = m*x + b
    
#     # Plot predictions
#     ax.plot(x, y_pred)
    
#     # Plot squared errors
#     xmin, xmax = ax.get_xlim()
#     ymin, ymax = ax.get_ylim()
#     scale = (xmax-xmin)/(ymax-ymin)
#     for x, y1, y2 in zip(x, y, y_pred):
#         bottom_left = (x, min(y1, y2))
#         height = abs(y1 - y2)
#         width = height * scale
#         ax.add_patch(Rectangle(xy=bottom_left, width=width, height=height, alpha=0.1))
    
#     # Print regression metrics
#     mse = mean_squared_error(y, y_pred)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y, y_pred)
#     r2 = r2_score(y, y_pred)
#     print('Mean Squared Error:', mse)
#     print('Root Mean Squared Error:', rmse)
#     print('Mean Absolute Error:', mae)
#     print('R^2:', r2)

In [0]:
def pred(series, coefs, intercept):
  return ((series.private_outdoor * coefs[0] + 
           series.bed_bath_dining * coefs[1] +
           intercept))

y_pred_train = train.apply(lambda row: pred(row, coefficients, intercept), axis=1)

y_pred_test = test.apply(lambda row: pred(row, coefficients, intercept), axis=1)
# y_pred

In [65]:
# TODO - redo Ryan's function, genericize standard metrics.

#RMSE
mse_train = mean_squared_error(y_train, y_pred_train)
mse_train
mse_test  = mean_squared_error(y_test,  y_pred_test )
mse_test

rmse_train = np.sqrt(mse_train)
rmse_test  = np.sqrt(mse_test )

#MAE
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test  = mean_absolute_error(y_test,  y_pred_test )

# R2
r2_train = r2_score(y_train, y_pred_train)
r2_test  = r2_score(y_test,  y_pred_test )

print(f'RMSE, training: ${rmse_train:,.2f}')
print(f'RMSE, test: ${rmse_test:,.2f}')
print()
print(f'MAE, training: ${mae_train:,.2f}')
print(f'MAE, test: ${mae_test:,.2f}')
print()
print(f'R^2, training: {(100 * r2_train):.2f}%')
print(f'R^2, test: {(100 * r2_test):.2f}%')


RMSE, training: $1,329.37
RMSE, test: $1,324.60

MAE, training: $894.27
MAE, test: $903.83

R^2, training: 43.09%
R^2, test: 43.55%


## Evaluate model so far.

- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!

**My best MAE so far is $903.83.**

**I have shared MAE and features on slack.**

## Push to GitHub.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

**Done.**

# More

#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !